In [1]:
import llm_core.llm as L
import os

In [ ]:
# try restricted vocab. Give it a set amount of allowed words to chose from to use the grammar with and see how well it performs
# Try with less restriction and see if results improve 
#
# Try monoligual instructions; すべてで日本語
# Try monolingual grammar lol

# turn offf ggrammar fileter
# try allowedchars ::= [\u3040-\u30FF, \u4e00 - \u9faf]

In [2]:
#os.environ['MODEL_PATH'] = "/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf"
LLM_GLOBAL_INSTANCE = L.Llama(
                n_ctx=1000,
                model_path=os.environ['MODEL_PATH'],
                n_gpu_layers=-1, verbose=0,
                # embedding=True
            )

In [6]:
# system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
# user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色を何をしてるのですか？"

In [ ]:
# No Kanji usage, rare katakana usage
grammar_string = r"""
root ::= sentence{1,2}
allowedchar ::= [\u3040-\u30FF]|[\u4e00-\u9faf]
word ::= allowedchar+
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.3,
    max_tokens=50,
    grammar=my_grammar
    )

{'id': 'chatcmpl-01a52087-a0ba-4210-ba4c-55548e7b686a',
 'object': 'chat.completion',
 'created': 1741368442,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '赤色が素晴らしいですです。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 52, 'completion_tokens': 7, 'total_tokens': 59}}

In [4]:
#With basic prompt, no character restrictions spamps \u0000 or \uffff
#Good responses with user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
#Talks in any vocab, but keeps sentences in が + です format. :D
# Try telling it to write with spaces between words?
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.3,
    max_tokens=50,
    grammar=my_grammar
    )

#Really good example sentence: '赤色は炎のように光ってます が 楽しく感じます です。'
#'赤色が一番目立つ色です。'

NameError: name 'system_input' is not defined

In [ ]:
# system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
# user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
system_input = "そのAがBですの文法だけで簡単な文章を書いてください"
user_input = "赤色は何をしってるのですか？"

In [36]:
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

training_data = list()
training_size = 5
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['赤色は目を引きつけます。جزاء الجملة باللغة العربية:\n\nالإجابة هي جملة منقوصة ولا تستوفي بنية جملة سليمة في اللغة العربية، لذا يمكن إعادة صي',
 '赤色は見るものを赤色として認識します。これは、色を表す説明です。より簡潔に表すなら、「赤色は視覚です」ですが、これで文法が少し傾きます。より正確',
 '赤色は見るものです。 \n\n（注：この文章は日本語の文法に則った簡素な表現です。文脈によっては、より自然な表現になる場合もあります。） \n\nもっと自然な表現としては：\n"赤',
 '赤色は視覚を何と言いますか？（補足情報を求める形式に変更しました） \n\nまたは\n\n赤色は物体の色を何と言いますか？ \n\nこのような形で質問に答えることができます。しかし',
 '赤色は目を引きつける効果があります。 \n\n（文法的には「赤色は目を引く」という文でも構いませんが、質問の意図に沿うように少し表現を変更しています']

In [4]:
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [ ]:
#only 1 or 2 allowedchar makes for much shorter+concise sentences, but necessarily correct (parts of words get cut off)
#adding more sentences makes them incoherent
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

#grammar works better with Japanese prompts
#English prompts are better without grammar
#Not much variety between responses though
#How does Llama grammar work??!!? Logits?

In [8]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['赤が喜です。',
 '赤が喜です。',
 '赤が火です。',
 '赤が喜です。',
 '赤が力です。',
 '赤が力です。',
 '赤が怒です。',
 '赤が喜です。',
 '赤が怒です。',
 '赤が光です。']

In [ ]:
# i and na-adj don't work well
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using i-adjectives to describe in the [word] が [adjective] form."
user_input = "太陽の景色を描いてください"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle adjective shii end
particle ::= "が"
adjective ::= [\u0000-\uffff]
shii ::= "しい"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [10]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['太陽が明るしい。',
 '太陽が輝かしい。',
 '太陽が明るしい。',
 '太陽が金色しい。',
 '太陽が真っしい。',
 '太陽が輝かしい。',
 '太陽が真っしい。',
 '太陽が金色しい。',
 '太陽が真っしい。',
 '太陽が金色しい。']

In [ ]:
# Needs a good system prompt to use しか at all
# Difficult to get it to use しか normally and not just repeat the user question with it.
#あんたは普通な人って想像してください。。何をできますか？ is a good prompt :D
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts using the しか～ない Japanese grammar pattern"
user_input = "あんたは普通な人って想像してください。。何をできますか？"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "しか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [12]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ],
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['あんたは普通な人って想像してください。何かできるのはほとんど何もしかありませんな。',
 'あんたは普通な人って想像してください。何か特別なことができるなんて思えないでしょう？しか選ばないなあ。',
 '彼は普通の人って想像してください。何か特別なことができるしかない samt な人です。',
 'あんたは普通な人って想像してください。そんな人には仕事の能力しかありませんな。',
 '彼は普通の人しか想像できません。何をできるかほとんど思い浮かびません。',
 'あんたは普通な人って想像してください。何かを達成するしかありませんね。',
 '彼は普通の人しか想像できないな。日常的なことしかできる気がする。料理をしたり、洗濯をしたり、掃除をしたりしかないよ。',
 'あんたは普通な人って想像してください。そんな人なら料理を作るのはもちろんです～ないですね。でも、仕事や学校のことを考えると、それ以外の特殊なスキルはあまり必要とされません～ないですよ。',
 'あなたは普通の人って想像してみてください。何か特別なスキルがあるのはしか考えられないな。',
 'あんたは普通な人って想像してください。何か特別なことをできるしかありませんな。']

In [13]:
# Responses might be better with grammar? More english junk, but the non-junk are shorter and more straight-forward
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using the かどうか grammar pattern"
user_input = "Ask me a quesiton in Japanese."
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "かどうか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [14]:
training_data = list()
training_size = 1000
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['日本の動物公园でパンダを見たかどうか、教えてくれない？（你在日本の動物公园看到熊猫了吗？） \n\nこれは日本の子供向けの本で使う質問の例です。このパターンは子供たちが自分の経',
 '日本語でお聞きBecome a question using the かどうか grammar pattern. \n\nあなたは今日お絵かきをするかどうか知りたいです。',
 "日本語でお尋ねしますね。この質問には、「かどうか」を使用してみました。\n\n絵本の主人公は、新しい友達ができたかどうか知りたいですか？（ Picture book's protagonist wants to know if they made a new",
 'あなたは今日公園に行くかどうか知っていますか？（あなたはきょうこうえんにい행을 가는かどうかしっていますか？） \n\nこの文は、今日公園にいくかどうかという質問を子供向けに簡潔',
 'あなたは今日学校に行きますか行かないか知りたいです。答え帮我填空吧，可以用 行きます or 行きません。例如：あなたは今日学校に行きます____。填空后句子的意思是“你今天',
 'あなたは今日公園に行くかどうか教えてください。',
 'あなたはお気に入りのキャラクターがいるかどうか教えてください。',
 'あなたは動物園に行きたいかどうか知りたいです。',
 'あなたは今天気-goodkah（ですか？）？（今日は天気が良い-ですか？） \n\nこの質問は、今日が良い天気かどうか尋ねています。',
 'あなたの favorite の本はなんかどうか教えてあげる？（あなたの favorite の本はなんですか？） \n\n(あなたの好きな本はなんですか？) この本は絵本でも小説でも何でもいいよ',
 'あなたは今日公園で遊ぶかどうか教えてください。',
 'あなたは今日公園に行こうかといますか？（あなたは今日公園に行こうかと思いますか？） \n\nこの質問は、"今日公園に行こうかと思いますか？"という形で書きました。"かどうか"という文',
 'あなたはりんごを食べますか、またはみかんを食べますか？（您是吃苹果还是吃橙子呢？） \n\nこの質問は、りんごかみかんのどちらを食べるかを',
 'あなたは猫が好きかどうか知りたいです。猫が好きですか？不喜欢猫的话，

In [15]:
import torch
import torch.nn as nn
from transformers.models.qwen2.modeling_qwen2 import Qwen2DecoderLayer
from transformers import Qwen2ForCausalLM, Qwen2Config
import transformers
from matplotlib import pyplot as plt
import gc

In [16]:
class IMDecoderLayer(nn.Module):
    mask = None
    vspace_to_emb = None
    emb_to_vspace = None

    def __init__(self, original_layer, emb_to_vspace, config):
        super().__init__()
        self.original_layer = original_layer

        if IMDecoderLayer.mask == None:
            IMDecoderLayer.mask = torch.zeros(config.vocab_size).to('cuda')

        if IMDecoderLayer.vspace_to_emb == None:
            IMDecoderLayer.vspace_to_emb =  nn.Linear(config.vocab_size, config.hidden_size).to('cuda')
        
        if IMDecoderLayer.emb_to_vspace == None:
            IMDecoderLayer.emb_to_vspace =  emb_to_vspace

    def forward(self, hidden_states, *args, **kwargs):
        hidden_states = self.original_layer(hidden_states, *args, **kwargs)
        hidden_states = hidden_states[0]

        # TODO: for inference, bypass this process on all tokens but the current one? Possible problem: maybe it will learn to rely on past tokens being allowed ones later in residual.
        # ^ maybe this process should only be done on last token if later note is a problem. If it isnt a problem, then it's preferable to do this b/c the layer will train faster.

        residual = hidden_states
        hidden_states = IMDecoderLayer.emb_to_vspace(residual)
        hidden_states *= IMDecoderLayer.mask
        hidden_states = IMDecoderLayer.vspace_to_emb(hidden_states)
        # hidden_states = (hidden_states + residual)/2 # NOTE: should also be normalization? -- avg: has to learn to recreate residual with affects added on -- add&norm: most things can be zero and only learns to affect some values
        hidden_states = hidden_states + residual # input layernorm means don't need to norm here? (and SHOULD norm after hidden state??)

        return (hidden_states,)

In [21]:
MODEL_NAME = 'Qwen/Qwen2.5-0.5B-Instruct'

config = Qwen2Config.from_pretrained(MODEL_NAME)

model = Qwen2ForCausalLM.from_pretrained(MODEL_NAME).to('cuda')

# FREEZE existing model. Only the new layer in IMDecoderLayer will be trained
for param in model.parameters():
    param.requires_grad = False

for i, _ in enumerate(model.model.layers):
    model.model.layers[i] = IMDecoderLayer(model.model.layers[i], model.lm_head, config)

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
optimizer = torch.optim.AdamW([p for p in IMDecoderLayer.vspace_to_emb.parameters() if p.requires_grad], lr=5e-5)

print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x IMDecoderLayer(
        (original_layer): Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [22]:
for sample in training_data:
    try:
        tokens = tokenizer(sample, return_tensors='pt', padding=True)
        tokens = {k:v.to('cuda') for k,v in tokens.items()}
        outputs = model(**tokens, labels=tokens['input_ids'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    except torch.OutOfMemoryError:
            print(f'OOM on {i}')
            try: del tokens, outputs
            except NameError: pass
            gc.collect()



In [28]:
tokens = tokenizer(['<User>: Write 3 example questions in Japanese.\n<Assistant>: '], return_tensors='pt', padding=True)
tokens = {k:v.to('cuda') for k,v in tokens.items()}

out = model.generate(
# out = model(
    **tokens,
    # labels=tokens['input_ids'],
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.001,
    do_sample=True,
    return_dict_in_generate=True,
    output_hidden_states=True
)
out_ids = out['sequences']
print(tokenizer.decode(out_ids[0]))

<User>: Write 3 example questions in Japanese.
<Assistant>: はいかじんに猫が好きかどうか教えてください。 

This question is asking if the child likes cats or not. It will help to understand whether a cat is interested in playing with toys or not. This will be helpful for children who
